In [2]:
import os
import pandas as pd
import json

import tensorflow as tf
from keras_tuner import Hyperband
from tensorflow import keras
from tensorflow.keras import layers
from tcn import TCN
from tensorflow.keras.layers import Dense, Input, concatenate
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping

## Data prep

In [3]:
import sys
sys.path.insert(1, '../')

from data_preparation import prepare_x_data, get_Y_labels, reshape_Y, reshape_X

In [4]:
scaler = StandardScaler()

### X data - features

In [5]:
# TRAIN data
x_train_visual = prepare_x_data('../Data/LLDs_video_openface/train',',', 5, scaler)
x_train_audio = prepare_x_data('../Data/LLDs_audio_eGeMAPS/train',';', 2, scaler) 

### Y data - labels

In [6]:
#Y TRAIN labels
y_train = get_Y_labels('../Data/labels_metadata.csv', 60, 164, scaler)
y_train = reshape_Y(y_train,len(x_train_visual),1,1)

# Tuner

In [7]:
def build_model(hp):
    # Define input layer.
    inputA = Input(shape=(None, 465))
    inputB = Input(shape=(None, 23))

    # Define hidden layer.
    tcn_v = TCN(nb_filters=hp.Int('v_nb_filters', min_value=1, max_value=465, step=1), 
                kernel_size=hp.Choice('v_kernel_size', [1,2,4,8,16,32,64,128]), 
                dilations=(128,256,512,1024,2048), 
                padding=hp.Choice('a_padding', ['causal', 'same']), 
                nb_stacks=hp.Choice('v_nb_stacks', [1,2]), 
                use_batch_norm=False, 
                use_layer_norm=False,
                use_weight_norm=False, 
                use_skip_connections=True, 
                dropout_rate=hp.Choice('v_dropout_rate', [1e-2, 1e-3, 1e-4]), 
                return_sequences=False, 
                input_shape=(None, 465))(inputA)

    tcn_a = TCN(nb_filters=hp.Int('a_nb_filters', min_value=1, max_value=23, step=1), 
                kernel_size=hp.Choice('a_kernel_size', [1,2,4,8,16,32,64,128]), 
                dilations=(128,256,512,1024,2048), 
                padding=hp.Choice('a_padding', ['causal', 'same']), 
                nb_stacks=hp.Choice('a_nb_stacks', [1,2]), 
                use_batch_norm=False, 
                use_layer_norm=False, 
                use_weight_norm=False,
                use_skip_connections=True, 
                dropout_rate=hp.Choice('a_dropout_rate', [1e-2, 1e-3, 1e-4]), 
                return_sequences=False, 
                input_shape=(None, 23))(inputB)


    # Define output layer.
    output_v = Dense(8, activation='relu')(tcn_v)
    output_a = Dense(8, activation='relu')(tcn_a)

    combined_input = concatenate([output_v, output_a])

    output = Dense(1)(combined_input)

    # Define optimizer and show summary.
    model = keras.Model(inputs=[inputA, inputB], outputs=[output])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), loss='mse', metrics=['mae'])

    return model

In [8]:
 # Define tuner. A new tuner is created for every file.
tuner = Hyperband(
    hypermodel=build_model, 
    objective='loss', 
    max_epochs=50, 
    factor=3, 
    # directory='TCN_Tuner', 
    project_name='TCN_Tuner_Multi'
)

early_stop = EarlyStopping(monitor='loss', patience=10)

INFO:tensorflow:Reloading Oracle from existing project ./TCN_Tuner_Multi/oracle.json


2022-05-21 22:07:55.412661: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-21 22:07:55.425685: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-21 22:07:55.915281: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:927] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-05-21 22:07:55.920659: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1660 Ti computeCapability: 7.5
coreClock: 1.59GHz coreCount: 24 deviceMemorySize: 6.00GiB deviceMemoryBandwidth: 268.26GiB/s
2022-05-21 22:07:55.920743: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-05-21 22:07:55.954921: I tensorflow/stream_executor/plat

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Reloading Tuner from ./TCN_Tuner_Multi/tuner0.json


In [9]:
# Tune the model one file at a time.
#Train visual data tuner
index = 0
for train_visual, train_audio in zip(x_train_visual, x_train_audio):
    train_visual = reshape_X(train_visual)
    train_audio = reshape_X(train_audio)

    tuner.search([train_visual, train_audio], y_train[index], epochs=10, callbacks=[early_stop]) 
    index += 1


Trial 92 Complete [00h 01m 31s]
loss: 9.467082023620605

Best loss So Far: 9.717570355860516e-05
Total elapsed time: 00h 17m 34s

Search: Running Trial #93

Hyperparameter    |Value             |Best Value So Far 
v_nb_filters      |370               |94                
v_kernel_size     |128               |128               
a_padding         |same              |same              
v_nb_stacks       |2                 |1                 
v_dropout_rate    |0.001             |0.01              
a_nb_filters      |4                 |4                 
a_kernel_size     |32                |1                 
a_nb_stacks       |1                 |2                 
a_dropout_rate    |0.001             |0.01              
learning_rate     |0.001             |0.001             
tuner/epochs      |6                 |6                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |2                 |2                 
tuner/round       |0                 |0      

2022-05-21 22:26:07.995053: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_0_bfc) ran out of memory trying to allocate 66.85MiB (rounded to 70092800)requested by op Fill
Current allocation summary follows.
2022-05-21 22:26:07.995138: I tensorflow/core/common_runtime/bfc_allocator.cc:972] BFCAllocator dump for GPU_0_bfc
2022-05-21 22:26:07.995153: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (256): 	Total Chunks: 316, Chunks in use: 316. 79.0KiB allocated for chunks. 79.0KiB in use in bin. 18.8KiB client-requested in use in bin.
2022-05-21 22:26:07.995158: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (512): 	Total Chunks: 8, Chunks in use: 8. 5.5KiB allocated for chunks. 5.5KiB in use in bin. 5.0KiB client-requested in use in bin.
2022-05-21 22:26:07.995162: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (1024): 	Total Chunks: 115, Chunks in use: 115. 146.2KiB allocated for chunks. 146.2KiB in use in bin. 122.7KiB client-req

ResourceExhaustedError: in user code:

    /home/anuh/anaconda3/envs/gpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/anuh/anaconda3/envs/gpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/anuh/anaconda3/envs/gpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/anuh/anaconda3/envs/gpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/anuh/anaconda3/envs/gpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/anuh/anacon

In [10]:
#Get df from all tuner searches
def get_hp_df(tunerdir):
    rootdir = tunerdir
    hp_values = []
    #Get learned hyperparameters for every file (trial) and append to a list
    for trial in os.listdir(rootdir):
        pathdir = os.path.join(rootdir, trial)
        filedir = os.path.join(pathdir, 'trial.json')
        
        if os.path.isdir(pathdir): #looking only for trial subfolders
            with open(filedir) as json_file:
                data = json.load(json_file)
                values = data['hyperparameters']['values'] #get learned hyperparameters
                hp_values.append(values) #append to list
        
    #Transform to dataframe
    hp_df = pd.DataFrame(hp_values)

    return hp_df

In [11]:
# Get average hyperparameter values for both modalities
hpdf_multi = get_hp_df('../TCN/TCN_Tuner_Multi')

In [12]:
print(""" Average hyperparameters for each data modality.
--------FOR BOTH MODAL DATA:""")
hpdf_multi.mean()

 Average hyperparameters for each data modality.
--------FOR BOTH MODAL DATA:


/tmp/ipykernel_6341/1505734101.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  hpdf_multi.mean()


v_nb_filters           212.586957
v_kernel_size           34.717391
v_nb_stacks              1.630435
v_dropout_rate           0.004248
a_nb_filters            11.413043
a_kernel_size           33.978261
a_nb_stacks              1.413043
a_dropout_rate           0.002702
learning_rate            0.003318
tuner/epochs            12.293478
tuner/initial_epoch      2.141304
tuner/bracket            2.336957
tuner/round              0.413043
dtype: float64